In [3]:
import pandas as pd

In [ ]:
df = pd.read_csv("hf://datasets/PrkhrAwsti/Twitter_Sentiment_3M/twitter_dataset.csv")

In [4]:
df.head()

,Unnamed: 0,tweet,sentiment
0,0,is upset that he can't update his Facebook by ...,0.0
1,1,@Kenichan I dived many times for the ball. Man...,0.0
2,2,my whole body feels itchy and like its on fire,0.0
3,3,"@nationwideclass no, it's not behaving at all....",0.0
4,4,@Kwesidei not the whole crew,0.0


In [20]:
dfsample = df.sample(n=100,random_state=100)

In [21]:
dfsample.to_csv('sample_para_dashboard_prueba.csv')

In [3]:
# Librerias necesarias
import torch
from transformers import RobertaTokenizer, RobertaForSequenceClassification
from transformers import Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
import pandas as pd


In [4]:
x = torch.rand(5, 3)
print(x)

tensor([[9.0223e-01, 4.5871e-01, 6.1152e-01],
        [8.1092e-01, 3.1761e-02, 6.0722e-02],
        [1.0787e-01, 4.2731e-04, 2.2336e-01],
        [4.3608e-01, 5.2794e-01, 5.6576e-01],
        [3.5234e-01, 5.1036e-01, 9.9494e-01]])


In [4]:
# Import required libraries
import torch
from torch.utils.data import DataLoader, TensorDataset
from transformers import RobertaForSequenceClassification
from torch.cuda.amp import autocast
from sklearn.metrics import classification_report, roc_auc_score
import time

# Step 1: Load the pre-trained RoBERTa model (without fine-tuned weights)
model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=2)

# Step 2: Set the model to evaluation mode
model.eval()

# Step 3: Move the model to the same device (GPU or CPU) as your fine-tuned model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Step 4: Load the validation data (updated path)
val_encodings, val_labels = torch.load(r'C:/Users/aless/Desktop/final project 2.1/val_encodings.pt')

# Step 5: Prepare the validation DataLoader
batch_size = 512
val_dataset = TensorDataset(val_encodings['input_ids'], val_encodings['attention_mask'], val_labels)
val_loader = DataLoader(val_dataset, batch_size=batch_size, num_workers=8)

# Step 6: Evaluate the pre-trained model on the validation set
# Initialize lists to store predictions and true labels
all_preds = []
all_labels = []
y_proba = []

# Start timer for inference
start_time = time.time()

# Disable gradient computation
with torch.no_grad():
    for step, batch in enumerate(val_loader):
        input_ids, attention_mask, labels = [x.to(device) for x in batch]

        # Enable mixed precision for faster computation
        with autocast():
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            logits = outputs.logits  # No need to index for CLS token, logits directly predict the classes

            # Predicted classes (0 for negative, 1 for positive)
            predictions = torch.argmax(logits, dim=-1)
            # Probabilities for ROC-AUC
            probabilities = torch.softmax(logits, dim=-1)[:, 1]  # Probabilities for class 1

        # Store predictions and true labels
        all_preds.extend(predictions.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())
        y_proba.extend(probabilities.cpu().numpy())

# Total inference time
total_time = time.time() - start_time
print(f"Total inference time: {total_time:.2f} seconds")

# Step 7: Calculate classification report and ROC-AUC score
print("Classification Report for Pre-trained RoBERTa Model:")
print(classification_report(all_labels, all_preds))

# Step 8: Calculate ROC-AUC score
roc_auc = roc_auc_score(all_labels, y_proba)
print(f"ROC-AUC Score: {roc_auc:.4f}")


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\aless\AppData\Local\Temp\ipykernel_9612\1183939198.py:20: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode un

Total inference time: 268.30 seconds
Classification Report for Pre-trained RoBERTa Model:
              precision    recall  f1-score   support

           0       0.50      1.00      0.67    235207
           1       0.55      0.00      0.00    234516

    accuracy                           0.50    469723
   macro avg       0.53      0.50      0.33    469723
weighted avg       0.53      0.50      0.34    469723

ROC-AUC Score: 0.5246
